In [1]:
library(tidyverse)
library(repr)
library(tidymodels)
options(repr.matrix.max.rows = 6) 

── Attaching packages ─────────────────────────────────────── tidyverse 1.3.1 ──

✔ ggplot2 3.3.6     ✔ purrr   0.3.4
✔ tibble  3.1.7     ✔ dplyr   1.0.9
✔ tidyr   1.2.0     ✔ stringr 1.4.0
✔ readr   2.1.2     ✔ forcats 0.5.1

── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()

── Attaching packages ────────────────────────────────────── tidymodels 1.0.0 ──

✔ broom        1.0.0     ✔ rsample      1.0.0
✔ dials        1.0.0     ✔ tune         1.0.0
✔ infer        1.0.2     ✔ workflows    1.0.0
✔ modeldata    1.0.0     ✔ workflowsets 1.0.0
✔ parsnip      1.0.0     ✔ yardstick    1.0.0
✔ recipes      1.0.1     

── Conflicts ───────────────────────────────────────── tidymodels_conflicts() ──
✖ scales::discard() masks purrr::discard()
✖ dplyr::filter()   masks stats::filter()
✖ recipes::fixed()  masks stringr::fixed()
✖ dplyr::lag()      masks stats::lag()
✖ yardstick::spec() masks r

In [46]:
brainstroke_data <- read_csv("https://drive.google.com/uc?export=download&id=1yBiO_qBE9_YBvnEyPe2bazH5ZCOBb1d6")

brainstroke_data

Rows: 4981 Columns: 11
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr (5): gender, ever_married, work_type, Residence_type, smoking_status
dbl (6): age, hypertension, heart_disease, avg_glucose_level, bmi, stroke

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


gender,age,hypertension,heart_disease,ever_married,work_type,Residence_type,avg_glucose_level,bmi,smoking_status,stroke
<chr>,<dbl>,<dbl>,<dbl>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<chr>,<dbl>
Male,67,0,1,Yes,Private,Urban,228.69,36.6,formerly smoked,1
Male,80,0,1,Yes,Private,Rural,105.92,32.5,never smoked,1
Female,49,0,0,Yes,Private,Urban,171.23,34.4,smokes,1
⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮
Female,45,1,0,Yes,Govt_job,Rural,95.02,31.8,smokes,0
Male,40,0,0,Yes,Private,Rural,83.94,30.0,smokes,0
Female,80,1,0,Yes,Private,Urban,83.75,29.1,never smoked,0


In [47]:
# changing chr data to categorical int data
# removing ever_married column
# changing Residence_type to residence_type
# change stroke and gender to categorical int data

brainstroke_data_v1 <- brainstroke_data |>
    mutate(gender = as_factor(ifelse(gender == 'Male', 0, 1)),
           smoking_status = as_factor(ifelse(smoking_status == 'never smoked', 0, 
                                      ifelse(smoking_status == 'formerly smoked', 1, 
                                      ifelse(smoking_status == 'smokes', 2, 
                                      ifelse(smoking_status == 'Unknown', 3, 'n/a'))))),
           work_type = as_factor(ifelse(work_type == 'Private', 0, 
                       ifelse(work_type == 'Self-employed', 1, 
                       ifelse(work_type == 'Govt_job', 2, 'n/a')))),
           Residence_type = as_factor(ifelse(Residence_type == 'Rural', 0, 1)),
           stroke = as_factor(stroke),
           heart_disease = as_factor(heart_disease),
           hypertension = as_factor(hypertension)) |>
    select(1:4, 6:11) 

colnames(brainstroke_data_v1)[6] <- 'residence_type'

brainstroke_data_v1

gender,age,hypertension,heart_disease,work_type,residence_type,avg_glucose_level,bmi,smoking_status,stroke
<fct>,<dbl>,<fct>,<fct>,<fct>,<fct>,<dbl>,<dbl>,<fct>,<fct>
0,67,0,1,0,1,228.69,36.6,1,1
0,80,0,1,0,0,105.92,32.5,0,1
1,49,0,0,0,1,171.23,34.4,2,1
⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮
1,45,1,0,2,0,95.02,31.8,2,0
0,40,0,0,0,0,83.94,30.0,2,0
1,80,1,0,0,1,83.75,29.1,0,0


In [ ]:
# summarized tables

brainstroke_data_v2 <- brainstroke_data_v1 